# Air Quality Analysis - Module Demo

This notebook demonstrates the `air_quality` modules developed for PM2.5 analysis.

## Modules Available:
- `air_quality.statistics` - Statistical analysis functions
- `air_quality.extremes` - Extreme value identification

We'll demonstrate each function with the air quality dataset.


## Setup

Import modules and load data:


In [60]:
import pandas as pd
import numpy as np

# Import our modules
from air_quality.statistics import (
    calculate_daily_mean,
    calculate_rolling_average,
    calculate_exceedance_count,
    calculate_aqi
)
from air_quality.extremes import (
    identify_extremes_threshold,
    identify_extremes_percentile,
    identify_consecutive_exceedances
)

# Load data
df = pd.read_csv('data/all_cities_pm25.csv', parse_dates=['date'])
print(f"Loaded {len(df)} observations")
print(f"\nCities: {df['city'].unique().tolist()}")
print(f"\nData preview:")
print(df.head())


Loaded 2196 observations

Cities: ['Los Angeles', 'Fresno', 'Phoenix', 'Denver', 'Salt Lake City', 'Pittsburgh']

Data preview:
        date         city pollutant      value
0 2024-01-01  Los Angeles     PM2.5  22.445674
1 2024-01-02  Los Angeles     PM2.5  13.680909
2 2024-01-03  Los Angeles     PM2.5   8.141040
3 2024-01-04  Los Angeles     PM2.5   6.034635
4 2024-01-05  Los Angeles     PM2.5  11.618910


## 1. Statistics Module Demo

### Function: `calculate_daily_mean()`
Calculates the mean of PM2.5 values using NumPy.


In [61]:
# Calculate mean PM2.5 for each city
print("Average PM2.5 by City:")
print("-" * 40)
for city in df['city'].unique():
    city_data = df[df['city'] == city]['value'].values
    mean_val = calculate_daily_mean(city_data)
    print(f"{city}: {mean_val:.2f} μg/m³")


Average PM2.5 by City:
----------------------------------------
Los Angeles: 10.87 μg/m³
Fresno: 10.03 μg/m³
Phoenix: 8.55 μg/m³
Denver: 6.64 μg/m³
Salt Lake City: 7.16 μg/m³
Pittsburgh: 7.73 μg/m³


### Function: `calculate_rolling_average()`
Computes a moving average with a specified window size.


In [62]:
# Calculate 7-day rolling average for all cities
print("7-Day Rolling Average (first non-NaN value for each city):")
print("-" * 60)

for city in df['city'].unique():
    city_data = df[df['city'] == city].sort_values('date')
    rolling_avg = calculate_rolling_average(city_data['value'].values, window=7)
    
    # Find first non-NaN value (day 7)
    first_valid_idx = 6  # 7th day (0-indexed)
    if first_valid_idx < len(rolling_avg):
        date_val = city_data['date'].iloc[first_valid_idx]
        orig_val = city_data['value'].iloc[first_valid_idx]
        roll_val = rolling_avg[first_valid_idx]
        print(f"{city:20s}: Day 7 ({date_val.strftime('%Y-%m-%d')}) "
              f"Original={orig_val:5.1f}, Rolling={roll_val:6.2f}")


7-Day Rolling Average (first non-NaN value for each city):
------------------------------------------------------------
Los Angeles         : Day 7 (2024-01-07) Original=  2.9, Rolling= 11.09
Fresno              : Day 7 (2024-01-07) Original=  7.1, Rolling= 11.86
Phoenix             : Day 7 (2024-01-07) Original=  5.5, Rolling= 21.02
Denver              : Day 7 (2024-01-07) Original=  5.3, Rolling=  9.04
Salt Lake City      : Day 7 (2024-01-07) Original=  1.1, Rolling= 12.39
Pittsburgh          : Day 7 (2024-01-07) Original=  8.5, Rolling=  7.11


### Function: `calculate_exceedance_count()`
Counts days exceeding a threshold (e.g., EPA daily standard of 35 μg/m³).


In [63]:
EPA_STANDARD = 35.0

print(f"Days Exceeding EPA Standard ({EPA_STANDARD} μg/m³):")
print("-" * 40)
for city in df['city'].unique():
    city_data = df[df['city'] == city]['value'].values
    count = calculate_exceedance_count(city_data, EPA_STANDARD)
    total = len(city_data)
    pct = (count/total*100) if total > 0 else 0
    print(f"{city}: {count} days ({pct:.1f}%)")


Days Exceeding EPA Standard (35.0 μg/m³):
----------------------------------------
Los Angeles: 3 days (0.8%)
Fresno: 6 days (1.6%)
Phoenix: 2 days (0.5%)
Denver: 0 days (0.0%)
Salt Lake City: 1 days (0.3%)
Pittsburgh: 0 days (0.0%)


### Function: `calculate_aqi()`
Converts PM2.5 concentration to EPA Air Quality Index with health categories.


In [64]:
# Demonstrate AQI calculation using actual city data
print("AQI Calculation for Selected Days from Dataset:")
print("-" * 70)
print(f"{'City':>15s} {'Date':>12s} {'PM2.5':>8s} {'AQI':>6s} {'Category':>35s}")
print("-" * 70)

# Get one example day from each city (the day with max PM2.5)
for city in df['city'].unique():
    city_data = df[df['city'] == city]
    max_idx = city_data['value'].idxmax()
    max_row = city_data.loc[max_idx]
    result = calculate_aqi(max_row['value'])
    print(f"{city:>15s} {max_row['date'].strftime('%Y-%m-%d'):>12s} "
          f"{max_row['value']:>8.1f} {result['aqi']:>6d} {result['category']:>35s}")


AQI Calculation for Selected Days from Dataset:
----------------------------------------------------------------------
           City         Date    PM2.5    AQI                            Category
----------------------------------------------------------------------
    Los Angeles   2024-07-05     40.3    113      Unhealthy for Sensitive Groups
         Fresno   2024-12-06     46.0    127      Unhealthy for Sensitive Groups
        Phoenix   2024-01-01     79.0    163                           Unhealthy
         Denver   2024-07-23     26.4     81                            Moderate
 Salt Lake City   2024-09-11     36.2    103      Unhealthy for Sensitive Groups
     Pittsburgh   2024-08-15     22.0     72                            Moderate


## 2. Extremes Module Demo

### Function: `identify_extremes_threshold()`
Identifies days where PM2.5 exceeds a specific threshold.


In [65]:
# Find extreme days for all cities using threshold method
print(f"Days Exceeding {EPA_STANDARD} μg/m³ by City:")
print("-" * 60)

for city in df['city'].unique():
    city_data = df[df['city'] == city].copy()
    # Rename columns for extremes module which expects 'Date' and 'PM2.5'
    city_data.rename(columns={'date': 'Date', 'value': 'PM2.5'}, inplace=True)
    extremes = identify_extremes_threshold(city_data, EPA_STANDARD)
    print(f"\n{city}: {len(extremes)} extreme days")
    if len(extremes) > 0:
        worst_day = extremes.iloc[0]
        print(f"  Worst day: {worst_day['Date'].strftime('%Y-%m-%d')} "
              f"(PM2.5 = {worst_day['PM2.5']:.1f} μg/m³)")


Days Exceeding 35.0 μg/m³ by City:
------------------------------------------------------------

Los Angeles: 3 extreme days
  Worst day: 2024-07-05 (PM2.5 = 40.3 μg/m³)

Fresno: 6 extreme days
  Worst day: 2024-12-06 (PM2.5 = 46.0 μg/m³)

Phoenix: 2 extreme days
  Worst day: 2024-01-01 (PM2.5 = 79.0 μg/m³)

Denver: 0 extreme days

Salt Lake City: 1 extreme days
  Worst day: 2024-09-11 (PM2.5 = 36.2 μg/m³)

Pittsburgh: 0 extreme days


### Function: `identify_extremes_percentile()`
Identifies extreme days based on percentile threshold (e.g., 95th percentile).


In [66]:
# Find 95th percentile extreme days for all cities
print(f"95th Percentile Extreme Days by City:")
print("-" * 60)

for city in df['city'].unique():
    city_data = df[df['city'] == city].copy()
    # Rename columns for extremes module
    city_data.rename(columns={'date': 'Date', 'value': 'PM2.5'}, inplace=True)
    percentile_extremes = identify_extremes_percentile(city_data, 95)
    print(f"\n{city}: {len(percentile_extremes)} extreme days")
    if len(percentile_extremes) > 0:
        worst_day = percentile_extremes.iloc[0]
        print(f"  Worst day: {worst_day['Date'].strftime('%Y-%m-%d')} "
              f"(PM2.5 = {worst_day['PM2.5']:.1f} μg/m³)")


95th Percentile Extreme Days by City:
------------------------------------------------------------

Los Angeles: 19 extreme days
  Worst day: 2024-07-05 (PM2.5 = 40.3 μg/m³)

Fresno: 19 extreme days
  Worst day: 2024-12-06 (PM2.5 = 46.0 μg/m³)

Phoenix: 19 extreme days
  Worst day: 2024-01-01 (PM2.5 = 79.0 μg/m³)

Denver: 19 extreme days
  Worst day: 2024-07-23 (PM2.5 = 26.4 μg/m³)

Salt Lake City: 19 extreme days
  Worst day: 2024-09-11 (PM2.5 = 36.2 μg/m³)

Pittsburgh: 19 extreme days
  Worst day: 2024-08-15 (PM2.5 = 22.0 μg/m³)


### Function: `identify_consecutive_exceedances()`
Finds consecutive sequences of days exceeding a threshold.


In [67]:
# Find consecutive exceedance periods for all cities
print(f"Consecutive Exceedance Periods (>{EPA_STANDARD} μg/m³):")
print("=" * 60)

for city in df['city'].unique():
    city_data = df[df['city'] == city].copy()
    # Rename columns for extremes module
    city_data.rename(columns={'date': 'Date', 'value': 'PM2.5'}, inplace=True)
    consecutive = identify_consecutive_exceedances(city_data, EPA_STANDARD)
    
    print(f"\n{city}: {len(consecutive)} period(s)")
    if len(consecutive) > 0:
        longest = consecutive.iloc[0]
        print(f"  Longest: {longest['duration']} days "
              f"({longest['start_date'].strftime('%Y-%m-%d')} to "
              f"{longest['end_date'].strftime('%Y-%m-%d')})")
        print(f"  Max PM2.5: {longest['max_pm25']:.1f} μg/m³")


Consecutive Exceedance Periods (>35.0 μg/m³):

Los Angeles: 3 period(s)
  Longest: 1 days (2024-07-05 to 2024-07-05)
  Max PM2.5: 40.3 μg/m³

Fresno: 1 period(s)
  Longest: 6 days (2024-12-03 to 2024-12-08)
  Max PM2.5: 46.0 μg/m³

Phoenix: 2 period(s)
  Longest: 1 days (2024-01-01 to 2024-01-01)
  Max PM2.5: 79.0 μg/m³

Denver: 0 period(s)

Salt Lake City: 1 period(s)
  Longest: 1 days (2024-09-11 to 2024-09-11)
  Max PM2.5: 36.2 μg/m³

Pittsburgh: 0 period(s)
